In [1]:
#####継続時間と離散時間の同時分析のための変量効果モデル#####
import numpy as np
import pandas as pd
import matplotlib.pyplot  as plt
import numpy.matlib
import scipy.linalg
import itertools
from scipy import sparse
from pandas.tools.plotting import scatter_matrix
from numpy.random import *
from scipy import optimize
import seaborn as sns
import time

#np.random.seed(20)

In [2]:
####データの発生####
##データの設定
hh = 10000   #ユーザー数
pt = np.random.poisson(np.random.gamma(15, 1/0.25, hh), hh)   #ユーザーあたりのレコード数
hhpt = np.sum(pt)   #総レコード数
dt = 200   #最大観測期間

In [3]:
##IDとインデックスの設定
#IDの設定
user_id = np.repeat(range(hh), pt)
pt_id = np.array(list(itertools.chain(*[np.array(range(pt[i]), dtype="int") for i in range(hh)])))

#インデックスの設定
index = np.array(range(hhpt))
user_index = [i for i in range(hh)]
for i in range(hh):
    user_index[i] = index[user_id==i]

In [4]:
####説明変数の生成####
##素性ベクトルの生成
k1 = 6; k2 = 8; k3 = 6
x1 = np.random.uniform(0, 1, hhpt*k1).reshape(hhpt, k1)
x2 = np.zeros((hhpt, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.5, 1)
    x2[:, j] = np.random.binomial(1, prob, hhpt)
x3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(2.0, k3)), hhpt)
x3 = np.delete(x3, np.sum(x3, axis=0).argmin(), axis=1)
x = np.hstack((np.repeat(1, hhpt).reshape(hhpt, 1), x1, x2, x3))

##個体間説明変数の生成
k1 = 4; k2 = 5; k3 = 5
u1 = np.random.normal(0, 1, hh*k1).reshape(hh, k1)
u2 = np.zeros((hh, k2))
for j in range(k2):
    prob = np.random.uniform(0.25, 0.5, 1)
    u2[:, j] = np.random.binomial(1, prob, hh)
u3 = np.random.multinomial(1, np.random.dirichlet(np.repeat(2.0, k3)), hh)
u3 = np.delete(u3, np.sum(u3, axis=0).argmin(), axis=1)
u = np.hstack((np.repeat(1, hh).reshape(hh, 1), u1, u2, u3))

In [5]:
####継続時間の応答変数を生成####
rp = 0
while True:
    rp = rp + 1

    ##パラメータの設定
    #モデルパラメータを生成
    alpha1 = np.log(np.random.uniform(0.7, 1.3, 1))
    beta1 = np.append(np.random.uniform(0, 1.2, 1), np.random.uniform(-0.6, 1.2, x.shape[1]-1))
    alphat1 = alpha1; betat1= beta1

    #変量効果を生成
    tau = np.random.uniform(0.5, 0.75, 1)
    theta = np.random.normal(0, tau, hh)   
    taut = tau; thetat = theta

    ##ワイブル分布からイベント時間を生成
    #ワイブル分布のを尺度パラメータを設定
    Lambda = np.exp(np.dot(x, beta1) + theta[user_id])

    #イベント時間を生成
    y_full = scipy.stats.weibull_min.rvs(np.exp(alpha1), 0, Lambda, hhpt)

    #停止条件
    if (np.max(y_full) < 1000) & (np.mean(y_full) > 5.0):
        break

In [6]:
##打ち切りの設定
y_list = [i for i in range(hh)]
z_list = [i for i in range(hh)]
x_list = [i for i in range(hh)]
user_id_list = [i for i in range(hh)]
pt_id_list = [i for i in range(hh)]
new_pt = np.repeat(0, hh)

for i in range(hh):
    #右側打ち切りのインデックスを作成
    y_dt = y_full[user_index[i]]
    index_censor = np.arange(pt[i])[np.cumsum(y_dt) < dt]

    #打ち切りパターンにより変数を設定
    if (pt[i] > len(index_censor)) & (len(index_censor) > 0):
        y_list[i] = np.append(y_dt[index_censor], dt - np.max(np.cumsum(y_dt[index_censor])))
        z_list[i] = np.append(np.repeat(0, np.max(index_censor) + 1), 1)
    elif len(index_censor)==pt[i]:
        y_list[i] = y_dt[index_censor]
        z_list[i] = np.repeat(0, pt[i])
    elif len(index_censor)==0:
        y_list[i] = np.array([dt])
        z_list[i] = np.array([1])

    #変数を格納
    n = len(y_list[i])
    index = user_index[i][np.arange(n)]
    new_pt[i] = n
    x_list[i] = x[index, ]
    user_id_list[i] = np.repeat(i, n) 
    pt_id_list[i] = np.arange(n)

In [7]:
#リストを変換
pt = new_pt
hhpt = np.sum(pt)
user_id = np.array(list(itertools.chain(*[user_id_list[i] for i in range(hh)])))
pt_id = np.array(list(itertools.chain(*[pt_id_list[i] for i in range(hh)])))
z = np.array(list(itertools.chain(*[z_list[i] for i in range(hh)])))
y1 = np.array(list(itertools.chain(*[y_list[i] for i in range(hh)])))
x1 = np.array(list(itertools.chain(*[x_list[i] for i in range(hh)])))
k1 = x1.shape[1]

In [8]:
#インデックスを設定
index = np.arange(hhpt)
user_index = [i for i in range(hh)]
for i in range(hh):
    user_index[i] = index[user_id==i]
z_index = index[z==1]
user_dt = sparse.coo_matrix((np.repeat(1, hhpt), (user_id, range(hhpt))), shape=(hh, hhpt)).tocsr()

In [10]:
####離散選択の応答変数を生成####
rp = 0
while True:
    rp = rp + 1

    ##データの結合
    u_vec = theta[user_id]
    x2 = np.hstack((x1, u_vec.reshape(hhpt, 1)))
    k2 = x2.shape[1]

    ##パラメータの生成
    beta2 = np.append(-0.7, np.random.normal(0, 0.75, k2-1))

    ##ベルヌーイ分布から応答変数を生成
    #ロジットと応答確率
    logit = np.dot(x2, beta2)
    prob = np.exp(logit) / (1 + np.exp(logit))

    #応答変数を生成
    y2 = np.random.binomial(1, prob, hhpt)

    #停止条件
    if (np.mean(y2) < 0.4) & (np.mean(y2) > 0.2):
        break

In [11]:
####マルコフ連鎖モンテカルロ法で継続時間と離散選択を同時推定####
##ワイブル比例ハザードモデルの対数事後分布
def LLho_w1(alpha1, beta1, beta_mu1, inv_Cov1, x1, y1, y_censorl, user_id):
    #形状パラメータの非負制約
    gamma = np.exp(alpha1)

    #ワイブルモデルの対数尤度
    Lambda = np.dot(x1, beta1) + theta[user_id]
    scale = (y_censorl - Lambda) / gamma
    LL_weibull = np.sum(z * (-np.log(gamma) + scale) - np.exp(scale))
    
    #多変量正規分布の対数事前分布
    er = beta1 - beta_mu1
    LL_mvn = np.dot(np.dot(er, inv_Cov1), er)
    
    #対数事後分布の和
    LL = LL_weibull + LL_mnn
    return LL

In [12]:
##ワイブル比例ハザードモデルのユーザーごとの対数尤度の和
def LLho_w2(alpha1, beta1, x1, y1, y_censorl, user_dt, user_id, hhpt):
    #形状パラメータの非負制約
    gamma = np.exp(alpha1)

    #ワイブルモデルの対数尤度
    Lambda = np.dot(x1, beta1) + theta[user_id]
    scale = (y_censorl - Lambda) / gamma
    LLi_weibull = z * (-np.log(gamma) + scale) - np.exp(scale)
    
    #正規分布の対数事前分布
    er = theta - theta_mu
    LLi_normal = np.power(er, 2) / tau
    
    LLi = np.array(np.dot(user_dt, sparse.csr_matrix(LLi_w.reshape(hhpt, 1))).todense()).reshape(-1) + LLi_normal
    return LLi

In [219]:
##ワイブル比例ハザードモデルのモデルパラメータの勾配ベクトル


array([0.14728253])

In [226]:
##事前分布の設定
#階層モデルの事前分布
theta_mu = 0
inv_tau = 1/tau

#モデルパラメータの事前分布
beta_mu1 = np.repeat(0, k1) 
inv_Cov1 = np.linalg.inv(np.diag(np.repeat(100, k1)))
beta_mu2 = np.repeat(0, k2) 
inv_Cov2 = np.linalg.inv(np.diag(np.repeat(100, k2)))

In [311]:
er = beta1 - beta_mu1
LL_mvn = -1/2 * np.dot(np.dot(er, inv_Cov1), er)

In [312]:
LL_mvn

-0.01913310090590226

In [319]:
cd = 0
for i in range(k1):
    cd = cd + (-1/2 * np.power(er[i], 2) / 100)

In [326]:
er

array([0.14728253])

In [325]:
er = alpha - 0
(-1/2 * np.power(er, 2) / 100)

array([-0.00010846])

In [321]:
alpha

array([0.14728253])